In [2]:
import ccxt
import time
from itertools import permutations

# Replace with your actual Bybit API keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
CLOSE_SPREAD_THRESHOLD = 0.8 / 100  # Close arbitrage at 0.80%
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade
TRADE_AMOUNT_USDT = 100  # Default trade amount in USDT


def get_tradable_tokens():
    """Fetches all tradable tokens on Bybit and filters out low-volume pairs"""
    try:
        markets = bybit.load_markets()
        tradable_tokens = []
        for symbol in markets:
            if '/USDT' in symbol and ':USDT' not in symbol:
                market_data = bybit.fetch_ticker(symbol)
                volume = market_data.get('quoteVolume', 0)
                if volume and volume >= MIN_VOLUME:
                    tradable_tokens.append(symbol)

        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens: {e}")
        return []


def get_price(symbol):
    """Fetch latest price for a given token if the market exists"""
    try:
        if symbol in bybit.load_markets():
            ticker = bybit.fetch_ticker(symbol)
            return float(ticker.get("last", ticker.get("lastPrice", 0)))
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None


def execute_trade(symbol, side, amount, require_permission=True):
    """Executes a trade on Bybit"""
    try:
        if require_permission:
            confirmation = input(f"⚠️ Confirm execution: {side.upper()} {amount} {symbol} (yes/no): ")
            if confirmation.lower() != 'yes':
                print("❌ Trade cancelled by user.")
                return

        order = bybit.create_order(
            symbol=symbol,
            type='market',
            side=side,
            amount=amount
        )
        print(f"✅ Trade executed: {order}")
    except Exception as e:
        print(f"❌ Error executing trade: {e}")


def check_intra_exchange_arbitrage(symbol):
    """Checks for spot vs futures arbitrage opportunities within Bybit"""
    spot_price = get_price(symbol)
    futures_symbol = symbol.replace('/USDT', '/USDT:USDT')

    if futures_symbol not in bybit.load_markets():
        return

    futures_price = get_price(futures_symbol)

    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (TRADING_FEE * 2)
    trade_amount = TRADE_AMOUNT_USDT / spot_price  # Convert USDT to token amount

    if net_profit >= SPREAD_THRESHOLD:
        print(f"🔍 Arbitrage Opportunity: {symbol}")
        print(f"    Spot: {spot_price:.2f} USDT | Futures: {futures_price:.2f} USDT")
        print(f"    Spread: {spread:.4%} | Net Profit: {net_profit:.4%}")

        execute_trade(symbol, 'buy', trade_amount)
        execute_trade(futures_symbol, 'sell', trade_amount)

    if spread >= CLOSE_SPREAD_THRESHOLD:
        print(f"🚀 Auto-closing Arbitrage Position for {symbol} as spread hit 0.80%")
        execute_trade(symbol, 'sell', trade_amount, require_permission=False)
        execute_trade(futures_symbol, 'buy', trade_amount, require_permission=False)


def check_triangular_arbitrage(tradable_tokens):
    """Checks for triangular arbitrage opportunities within Bybit"""
    for route in permutations(tradable_tokens, 3):
        base_asset = route[0].split('/')[0]
        final_asset = route[2].split('/')[1]
        
        if base_asset == final_asset:
            try:
                first_leg_price = get_price(route[0])
                second_leg_price = get_price(route[1])
                third_leg_price = get_price(route[2])

                if first_leg_price and second_leg_price and third_leg_price:
                    amount_after_first_trade = (TRADE_AMOUNT_USDT / first_leg_price) * second_leg_price
                    amount_after_second_trade = (amount_after_first_trade / second_leg_price) * third_leg_price
                    final_amount = amount_after_second_trade
                    net_expected_return = final_amount - (TRADING_FEE * 3)

                    if net_expected_return > 1.005:
                        print(f"✅ Triangular Arbitrage: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                        execute_trade(route[0], 'buy', TRADE_AMOUNT_USDT / first_leg_price)
                        execute_trade(route[1], 'buy', amount_after_first_trade)
                        execute_trade(route[2], 'sell', amount_after_second_trade)
            except Exception:
                continue


while True:
    try:
        bybit_tokens = get_tradable_tokens()
        if not bybit_tokens:
            time.sleep(10)
            continue

        for token in bybit_tokens:
            check_intra_exchange_arbitrage(token)

        check_triangular_arbitrage(bybit_tokens)
        time.sleep(CHECK_INTERVAL)
    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)

🔍 Arbitrage Opportunity: OM/USDT
    Spot: 6.64 USDT | Futures: 6.77 USDT
    Spread: 1.9607% | Net Profit: 1.7607%


⚠️ Confirm execution: BUY 15.07068149621726 OM/USDT (yes/no):  yes


✅ Trade executed: {'info': {'orderId': '1910142870661371392', 'orderLinkId': '1910142870661371393'}, 'id': '1910142870661371392', 'clientOrderId': '1910142870661371393', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'OM/USDT', 'type': None, 'timeInForce': None, 'postOnly': None, 'reduceOnly': None, 'side': None, 'price': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'amount': None, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'fees': [], 'stopPrice': None}


⚠️ Confirm execution: SELL 15.07068149621726 OM/USDT:USDT (yes/no):  yes


✅ Trade executed: {'info': {'orderId': '54df5d03-af64-4d7d-8a00-520b4a37748a', 'orderLinkId': ''}, 'id': '54df5d03-af64-4d7d-8a00-520b4a37748a', 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'OM/USDT:USDT', 'type': None, 'timeInForce': None, 'postOnly': None, 'reduceOnly': None, 'side': None, 'price': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'amount': None, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'fees': [], 'stopPrice': None}
🚀 Auto-closing Arbitrage Position for OM/USDT as spread hit 0.80%
✅ Trade executed: {'info': {'orderId': '1910143119366821376', 'orderLinkId': '1910143119366821377'}, 'id': '1910143119366821376', 'clientOrderId': '1910143119366821377', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'OM/USDT', 'type': None, 'timeInForce': None